In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import requests

from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from tensorflow.keras import models, layers, optimizers, regularizers
from keras import models
from keras import layers
from keras import backend
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df1 = pd.read_csv('train_1.csv', encoding='iso-8859-1')
df2 = pd.read_csv('train_2.csv', encoding='iso-8859-1')
df3 = pd.read_csv('test_1.csv', encoding='iso-8859-1')
df4 = pd.read_csv('test_2.csv', encoding='iso-8859-1')


In [ ]:
value_column = ["online_time", "1_total_fee", "2_total_fee", "3_total_fee", "4_total_fee", "month_traffic",  "contract_time", "pay_times", "last_month_traffic", "local_trafffic_month", "local_caller_time", "service1_caller_time",  "service2_caller_time", "age", "former_complaint_num", "former_complaint_fee" ]

categorical_column = [ "current_service", "service_type", "is_mix_service", "many_over_bill", "contract_type",  "is_promise_low_consume", "net_service", "pay_num", "gender",  "complaint_level",  ]
categorical_column_without_trash_col =  [ "current_service", "service_type", "is_mix_service", "many_over_bill", "contract_type",  "is_promise_low_consume",  "complaint_level",  ]

In [ ]:
# Объединение всех таблиц в один DataFrame
df_combined = pd.DataFrame(pd.concat([df1, df2, df3, df4], axis=0, ignore_index=True))
# for i in range(len(df_combined)):
#     for j in range(len(df_combined[0])):
#         if df_combined[i][j] == None:
#             df_combined = 0.5
# df_combined = pd.DataFrame(df_combined)
print(df_combined)
del df_combined["net_service"]
del df_combined["pay_num"]
del df_combined["user_id"]
del df_combined["ï»¿service_type"]
df_combined.replace("\\N", np.nan, inplace=True)
df_combined.replace("NaN",np.nan, inplace=True)
df_combined = df_combined.dropna()

print(df_combined)

In [ ]:

df_combined = df_combined[(df_combined['gender'] == 1) | (df_combined['gender'] == 2) ]
# Приведение столбца gender к типу int
df_combined['gender'] = df_combined['gender'].astype(int)
df_combined['gender'] = pd.factorize(df_combined['gender'])[0]


names = ["2_total_fee", "gender", "3_total_fee", "current_service", "service_type"]


# Создание экземпляра OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

for name in categorical_column_without_trash_col:
 # Применение OneHotEncoder к целевому столбцу
 encoded_columns = encoder.fit_transform(df_combined[[name]])
 
 # Преобразование закодированных данных в DataFrame
 encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out([name]))
 
 # Объединение закодированных столбцов с исходным DataFrame (удалив при этом исходный столбец)
 df = df_combined.drop(name, axis=1)
 df_combined = pd.concat([df, encoded_df], axis=1)


del df_combined["complaint_level_nan"]
del df_combined["contract_type_nan"]
del df_combined["many_over_bill_nan"]
del df_combined["is_mix_service_nan"]
del df_combined["service_type_nan"]

df_combined.replace("\\N", np.nan, inplace=True)
df_combined.replace("NaN",np.nan, inplace=True)
df_combined = df_combined.dropna()

print(df_combined)

print(df_combined['gender'].value_counts())



In [ ]:
values = df_combined['gender']
points = df_combined.drop(['gender'], axis=1)
train_points, test_points, train_values, test_values = train_test_split(points, values, test_size = 0.2)

In [ ]:
rf_model = ensemble.RandomForestClassifier(n_estimators=50)
rf_model.fit(train_points, train_values)
test_predict_rf = rf_model.predict(test_points)
print(accuracy_score(test_values, test_predict_rf))


In [ ]:
model = models.Sequential()
model.add(layers.Dense(32, activation="relu", input_shape=(train_points.shape[1],)))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

# Компиляция модели с измененной скоростью обучения
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Обучение модели
results = model.fit(
    train_points, train_values,
    epochs=10,
    batch_size=100,
    validation_data=(test_points, test_values)
)

# Проверка результатов
print("Test-Accuracy:", np.max(results.history["accuracy"]))

In [ ]:
model = models.Sequential()
model.add(layers.Dense(32, activation="relu", input_shape=(train_points.shape[1],), kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(32, activation="relu", kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.Dense(1, activation="sigmoid"))

optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

results = model.fit(
    train_points, train_values,
    epochs=5,
    batch_size=100,
    validation_data=(test_points, test_values)
)

print("Test-Accuracy:", np.max(results.history["accuracy"]))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, zscore
from scipy import stats

# Проверка каждого столбца
for column in df_combined.columns:
    if df[column].dtype in ['int64', 'float64']:
        print(f'--- {column} ---')
        
        # Гистограмма и плотность распределения
        plt.figure(figsize=(10, 6))
        sns.histplot(df[column], kde=True)
        plt.title(f'Histogram and Density Plot of {column}')
        plt.show()
        
        # Q-Q Plot
        plt.figure(figsize=(10, 6))
        stats.probplot(df[column], dist="norm", plot=plt)
        plt.title(f'Q-Q Plot of {column}')
        plt.show()
        
        # Тест Шапиро-Уилка
        stat, p = shapiro(df[column])
        print('Statistics=%.3f, p=%.3f' % (stat, p))
        if p > 0.05:
            print('Sample looks Gaussian (fail to reject H0)')
        else:
            print('Sample does not look Gaussian (reject H0)')
        
        # Boxplot
        plt.figure(figsize=(10, 6))
        sns.boxplot(df[column])
        plt.title(f'Boxplot of {column}')
        plt.show()
        
        # Z-Score
        z_scores = zscore(df[column])
        outliers = np.where(np.abs(z_scores) > 3)
        print("Outliers at indices:", outliers)
        
        # IQR
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[column][(df[column] < (Q1 - 1.5 * IQR)) | (df[column] > (Q3 + 1.5 * IQR))]
        print("Outliers:", outliers)